In [ ]:
import soundfile as sf

FILE_PATH = '/Users/admin/Downloads/9-Audio 1 [2020-11-10 232021].aif'
# FILE_PATH = '/Users/admin/Downloads/Clap 2 [2020-04-24 220633].aif'

sf.info(FILE_PATH, verbose=False)

In [ ]:
audio_file = sf.SoundFile(FILE_PATH)
signal_array = audio_file.read()
l_channel = signal_array[:,0]
r_channel = signal_array[:,1]
audio_length = audio_file.frames/audio_file.samplerate
audio_length

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

time_axis = np.linspace(0, audio_length, num=l_channel.shape[0])

plt.figure(figsize=(15, 5))
plt.plot(time_axis, l_channel)
plt.title('Left Channel')
plt.ylabel('Signal Value')
plt.xlabel('Time (s)')
plt.xlim(0, audio_length)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

time_axis = np.linspace(0, audio_length, num=r_channel.shape[0])

plt.figure(figsize=(15, 5))
plt.plot(time_axis, r_channel)
plt.title('Left Channel')
plt.ylabel('Signal Value')
plt.xlabel('Time (s)')
plt.xlim(0, audio_length)
plt.show()

In [ ]:
import math



def get_quadratic_kernel(kernel_size):
    kernel = np.array(range(kernel_size))
    kernel = kernel - math.floor(kernel_size / 2)
    kernel = kernel / kernel_size
    kernel = np.ma.power(kernel, 2)
    kernel = kernel * -1
    kernel = kernel + abs(np.ma.min(kernel))
    return kernel    


def get_ones_kernel(kernel_size):
    return np.ones(kernel_size) / kernel_size


def smooth_audio_data(audio_data, resample_ratio, kernel, final_resample_ratio=30):
    absolute_audio_data = np.absolute(audio_data)
    resampled_audio_data = np.interp(
        np.arange(0, len(absolute_audio_data), resample_ratio), 
        np.arange(0, len(absolute_audio_data)), 
        absolute_audio_data
    )
    convolved_data = np.convolve(resampled_audio_data, kernel, mode='same')
    
    resampled_convolved_data = np.interp(
        np.arange(0, len(convolved_data), final_resample_ratio), 
        np.arange(0, len(convolved_data)), 
        convolved_data
    )
    
    # normilised = (resampled_convolved_data-np.min(resampled_convolved_data))/(np.max(resampled_convolved_data)-np.min(resampled_convolved_data))
    return resampled_convolved_data

resample_ratio = audio_file.samplerate / 1000
ones_kernel = get_ones_kernel(35)
convolved_l_channel_20 = smooth_audio_data(l_channel, resample_ratio, ones_kernel, 20)
convolved_l_channel_30 = smooth_audio_data(l_channel, resample_ratio, ones_kernel, 30)
convolved_l_channel_40 = smooth_audio_data(l_channel, resample_ratio, ones_kernel, 40)

plt.figure(figsize=(15, 5))
plt.plot(
    np.linspace(0, audio_length, num=convolved_l_channel_20.shape[0]), 
    convolved_l_channel_20,
    label='kernel 20'
)
plt.plot(
    np.linspace(0, audio_length, num=convolved_l_channel_30.shape[0]), 
    convolved_l_channel_30,
    label='kernel 30'
)

plt.plot(
    np.linspace(0, audio_length, num=convolved_l_channel_40.shape[0]), 
    convolved_l_channel_40,
    label='kernel 40'
)

plt.title('Left Channel')
plt.ylabel('Signal Value')
plt.xlabel('Time (s)')
plt.legend()
plt.xlim(0, audio_length)
plt.show()